# Fine-Tuning Models
> Fine-tuning using your own data

In this notebook, we'll use two references:https://huggingface.co/transformers/custom_datasets.html as a guide for our work.  We'll use the HuggingFace dataset we've already created and use it directly!

### Install required packages
Note that this is mostly required if you're on Google Colab.

In [ ]:
#! pip install transformers
#! pip install datasets

### Import packages of interest

In [ ]:
import numpy as np
import pandas as pd

from datasets import load_dataset, load_metric, Dataset
from transformers import pipeline
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from huggingface_hub import notebook_login

# 0. Log into HuggingFace CLI
Why are we doing this? Below, we'll use our own user accounts to grab datasets and upload models. If we don't do this, we'll have to pass in the auth token over. This isn't bad, but let's streamline our efforts!

In [ ]:
#!git config credential.helper store

In [ ]:
#notebook_login()

# 1. Load data from HuggingFace Hub or from disk

In [ ]:
ds_path = 'charreaubell/demo_data'
demo_ds = load_dataset(ds_path, use_auth_token=True)

Downloading:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

Using custom data configuration charreaubell--demo_data-e1481c242d53578c


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/7.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.64k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /Users/bellcs1/.cache/huggingface/datasets/parquet/charreaubell--demo_data-e1481c242d53578c/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# 2. Pre-process inputs
What's a tokenizer and what does it do? Let's learn more using Huggingface's [instruction on tokenizers](https://huggingface.co/course/chapter2/4?fw=pt). Then, let's try it on our own!

In [ ]:
#instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
tokenizer.name_or_path

'distilbert-base-cased'

In [ ]:
#define tokenizing function
def tokenize_inputs(example):
    return tokenizer(example['text'], truncation = True)

In [ ]:
#do the tokenizing using map function
tokenized_ds = demo_ds.map(tokenize_inputs, batched=True,
                           remove_columns = ['age', 'article_id', 'college_major',
                                             'first_name', 'last_name', 'years_of_journalism',
                                             'text'])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_ds

DatasetDict({
    valid: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 4
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 4
    })
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 12
    })
})

What's this `truncation` argument and this `batched` argument? Let's take a look.

## An aside on dynamically padded batch size
HF has the capacity to dynamically pad your batches such that each input is only as long as any given input in the batch. This helps with memory.You can learn more [here](https://huggingface.co/course/chapter3/2?fw=pt). For now, we'll simply instantiate a data collator and use it during training to demonstrate how we can do this.

In [ ]:
#Instantiate data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. Train model

In [ ]:
#recall that the dataset carries information about the classes
demo_ds['train'].features['label']

ClassLabel(num_classes=5, names=['engineering', 'humanities', 'prelaw', 'premed', 'science'], names_file=None, id=None)

In [ ]:
#get the number of classes and label conversions
no_classes = demo_ds['train'].features['label'].num_classes
id2label = {ind:label for ind, label in enumerate(demo_ds['train'].features['label'].names)}
label2id = {label:ind for ind, label in id2label.items()}

In [ ]:
#check it out
print(id2label)
label2id

{0: 'engineering', 1: 'humanities', 2: 'prelaw', 3: 'premed', 4: 'science'}


{'engineering': 0, 'humanities': 1, 'prelaw': 2, 'premed': 3, 'science': 4}

## Define model and task architecture

In [ ]:
# Choose the model type and instantiate it for the task
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased",
                                                           num_labels=no_classes,
                                                           id2label=id2label,
                                                           label2id=label2id)
model.name_or_path

## Define settings for basic model training and train

In [ ]:
#set training arguments
training_args = TrainingArguments("test_trainer",
                                 logging_strategy='epoch')

#setup training loop with arguments
trainer = Trainer(model=model,
                  args=training_args,
                  tokenizer=tokenizer,
                  data_collator=data_collator,
                  train_dataset=tokenized_ds['train'],
                  eval_dataset=tokenized_ds['valid'])

#train
trainer.train()

***** Running training *****
  Num examples = 12
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6


Step,Training Loss
2,1.603900
4,1.535800
6,1.500800




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=6, training_loss=1.546820878982544, metrics={'train_runtime': 12.0564, 'train_samples_per_second': 2.986, 'train_steps_per_second': 0.498, 'total_flos': 928356357240.0, 'train_loss': 1.546820878982544, 'epoch': 3.0})

### Reflect and Discuss
* How many epochs of training did this undergo? Why do you think it stopped at this number of epochs?
* What if you wanted to train the model more? How do you think you could change the number of epochs?
* Practically speaking, how is the model performing?

## Training with performance metrics

In [ ]:
#load a metric
metric = load_metric("accuracy")

#define the metric behavior
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [ ]:
#set new training arguments
training_args = TrainingArguments("test-trainer",
                                  logging_strategy = "epoch",
                                  evaluation_strategy="epoch",
                                  report_to='all')

#setup training loop
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['valid'],
    compute_metrics=compute_metrics
)

#train model
trainer.train()

PyTorch: setting up devices
***** Running training *****
  Num examples = 12
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6


Epoch,Training Loss,Validation Loss,Accuracy
1,1.396100,1.622399,0.250000
2,1.340900,1.647581,0.000000
3,1.183700,1.661458,0.000000


***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=6, training_loss=1.3068776925404866, metrics={'train_runtime': 13.3811, 'train_samples_per_second': 2.69, 'train_steps_per_second': 0.448, 'total_flos': 928356357240.0, 'train_loss': 1.3068776925404866, 'epoch': 3.0})

### Reflect and Discuss
* What new observations are present during model training?
* What comments can you make on the performance of the model now?
* What metrics are appropriate for your application?
* Consider that model training is done in-memory (the model weights are updated in memory, but not returned), and both of our `Trainer`s trained our model `model`. After basic training from Step 9 and training from Step 10, how many epochs has the model been trained?
* What are some advantages and disadvantages of the in-memory training?

## A brief aside on performance metrics
You may want to use other performance metrics than accuracy. Here are some [metrics available through Huggingface](https://huggingface.co/metrics). If you check out the metrics folder on the [Huggingface datasets](https://github.com/huggingface/datasets) repository, you'll be able to see what's necessary if you need to define another metric. Let's try a different metric!

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    #get predictions by using index of max logit
    predictions = np.argmax(logits, axis=-1)
    
    #calculate classification report
    perfs = precision_recall_fscore_support(labels, predictions, average='macro', zero_division=0)
    perf_dict = dict(zip(['precision', 'recall', 'fscore', 'support'], perfs))
    
    #return dictionary
    return perf_dict

In [ ]:
#setup training loop
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['valid'],
    compute_metrics=compute_metrics
)

trainer.train()

***** Running training *****
  Num examples = 12
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6


Epoch,Training Loss,Validation Loss,Precision,Recall,Fscore,Support
1,0.989800,1.758532,0.000000,0.000000,0.000000,None
2,0.965100,1.816048,0.000000,0.000000,0.000000,None
3,0.765700,1.820976,0.000000,0.000000,0.000000,None


***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=6, training_loss=0.9068659941355387, metrics={'train_runtime': 13.4469, 'train_samples_per_second': 2.677, 'train_steps_per_second': 0.446, 'total_flos': 928356357240.0, 'train_loss': 0.9068659941355387, 'epoch': 3.0})

## A brief aside on model training
One of several points of ambiguity when training models is how long should they train for? A way to approach this is to monitor the models and run them repeatedly, starting from the last checkpoint. Another way is through training a number of epochs (if you model trains quickly enough) and then always load the best model according to some metric at the end. Let's take a look at this.

We can realize this through `TrainingArguments`!

In [ ]:
#set new training arguments
training_args = TrainingArguments("test-trainer",
                                  overwrite_output_dir=True,
                                  logging_strategy = "epoch",
                                  evaluation_strategy="epoch",
                                  save_strategy='epoch',
                                  load_best_model_at_end = True,
                                  metric_for_best_model='fscore',
                                  greater_is_better=True,
                                  report_to='all')

#setup training loop
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['valid'],
    compute_metrics=compute_metrics
)

#train model
trainer.train()

PyTorch: setting up devices
***** Running training *****
  Num examples = 12
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6


Epoch,Training Loss,Validation Loss,Precision,Recall,Fscore,Support
1,1.603700,1.566911,0.166667,0.333333,0.222222,None
2,1.537900,1.555614,0.166667,0.333333,0.222222,None
3,1.492800,1.553270,0.166667,0.333333,0.222222,None


***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-2
Configuration saved in test-trainer/checkpoint-2/config.json
Model weights saved in test-trainer/checkpoint-2/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-2/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-2/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-4
Configuration saved in test-trainer/checkpoint-4/config.json
Model weights saved in test-trainer/checkpoint-4/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-4/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-4/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-6
Configuration saved in test-trainer/checkpoint-6/config.json
Model w

TrainOutput(global_step=6, training_loss=1.5448025067647297, metrics={'train_runtime': 16.4141, 'train_samples_per_second': 2.193, 'train_steps_per_second': 0.366, 'total_flos': 928356357240.0, 'train_loss': 1.5448025067647297, 'epoch': 3.0})

# 4. Using trained model with `Trainer`
## Evaluate

In [ ]:
trainer.evaluate(tokenized_ds['train'])

***** Running Evaluation *****
  Num examples = 12
  Batch size = 8


{'eval_loss': 1.5361217260360718,
 'eval_precision': 0.06666666666666667,
 'eval_recall': 0.2,
 'eval_fscore': 0.1,
 'eval_support': None,
 'eval_runtime': 1.0496,
 'eval_samples_per_second': 11.433,
 'eval_steps_per_second': 1.905,
 'epoch': 3.0}

## Predict

In [ ]:
trainer.predict(tokenized_ds['train'])

***** Running Prediction *****
  Num examples = 12
  Batch size = 8


PredictionOutput(predictions=array([[-6.0932573e-02,  2.5887752e-01,  2.0878818e-02, -1.0038625e-02,
         4.1899920e-02],
       [-7.0849620e-02,  2.6222461e-01,  3.7979402e-02, -7.9516068e-02,
         7.8266852e-02],
       [-6.0193807e-02,  2.6336738e-01,  3.2106131e-02, -7.4920192e-02,
         3.0398801e-02],
       [-6.9419034e-02,  2.2632037e-01,  5.2801825e-02, -6.1321128e-02,
         4.6445504e-02],
       [-5.5189542e-02,  2.4058674e-01,  4.0477935e-02, -3.9512221e-02,
         4.5452416e-03],
       [-5.4554872e-02,  2.2160059e-01,  1.0810974e-01, -8.2729846e-02,
         6.7920238e-04],
       [-8.0790840e-02,  2.3849653e-01,  7.1875341e-03, -7.5528726e-02,
         1.2777004e-01],
       [-6.0694143e-02,  2.8064272e-01,  2.6836798e-02, -5.2206773e-02,
         3.5131097e-02],
       [-5.4284588e-02,  2.8006762e-01,  1.9655362e-02, -3.9448284e-02,
         3.0258402e-02],
       [-5.6016058e-02,  2.2985230e-01,  1.1007115e-04, -7.0286907e-02,
         1.0518595e-01],
 

# 5. Sharing and saving your model
## Using `Trainer`
During training and using the Trainer class, you can also upload your model directly to HuggingFace Hub as it trains. Read more about this process on the [HF course documentation](https://huggingface.co/course/chapter4/3?fw=pt).

Let's check out how to do this. It's as simple as modifying our `TrainingArguments`! Don't forget to have already logged in using your authorization token or use the `use_auth_token` paramter to access your HF account. You'll need to have git-lfs installed to use this feature, so if you're on Google Colab, you can execute the line below. You can also `conda install -c conda-forge git-lfs` if you're using a conda environment.

In [ ]:
#!apt-get install git-lfs

In [ ]:
#set new training arguments
training_args = TrainingArguments("test-trainer",
                                  overwrite_output_dir=True,
                                  logging_strategy = "epoch",
                                  evaluation_strategy="epoch",
                                  save_strategy='epoch',
                                  push_to_hub=True,
                                  hub_model_id='charreaubell/distilbert-magazine-classifier',
                                  report_to='all')

#setup training loop
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['valid'],
    compute_metrics=compute_metrics
)

#train model
trainer.train()

#### Reflect
Visit your repository and take a look to make sure your model uploaded. Answer the following questions:
* Where did your model save locally (directory)?
* What are the contents of the saved model?
* Investigate your uploaded model.

In [ ]:
#it's recommended to push the final version to HF after training completes.
trainer.push_to_hub(commit_message='end of training 3 epochs')

#### Reflect
Visit your repository once more (you'll likely need to refresh) and check out the changes.
* What is different from the uploads during training?
* What do you observe about the model cards?

## Fine-grained save/push access
You can also push the model and/or tokenizer directly using the `push_to_hub` methods in their classes. You can learn more about this [in the Huggingface docs.](https://huggingface.co/course/chapter4/3?fw=pt) An example of using trainer to do this is shown below.

In [ ]:
trainer.save_model('test-trainer')

Saving model checkpoint to test-trainer
Configuration saved in test-trainer/config.json
Model weights saved in test-trainer/pytorch_model.bin
tokenizer config file saved in test-trainer/tokenizer_config.json
Special tokens file saved in test-trainer/special_tokens_map.json


# 6. Using your fine-tuned model

In [ ]:
#create pipeline from your classifier
mag_classifier = pipeline('text-classification', model='test-trainer', use_auth_token=True)

#get output
mag_class = mag_classifier('The cat is prettier than any cat I have ever seen.')

In [ ]:
mag_class

[{'label': 'humanities', 'score': 0.2947634756565094}]